In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import load_model

# Cargar el modelo entrenado
model = load_model('model_maxv2.h5')

class_indices = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
classes = list(class_indices.keys())

cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Leer el frame de la cámara
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Convertir el frame a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detectar rostros en el frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Realizar predicciones solo si se detecta al menos un rostro
    for (x, y, w, h) in faces:
        # Extraer el rostro detectado
        face = frame[y:y+h, x:x+w]
        
        # Redimensionar el rostro al tamaño requerido por el modelo
        img = cv2.resize(face, (299, 299))
        
        # Preprocesar la imagen
        img = preprocess_input(img)
        img = np.expand_dims(img, axis=0)
        
        # Hacer la predicción
        predictions = model.predict(img)
        predicted_class = classes[np.argmax(predictions)]
        
        # Mostrar la clase predicha en el frame
        cv2.putText(frame, predicted_class, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        # Dibujar un rectángulo alrededor del rostro detectado
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Mostrar el frame
    cv2.imshow('Camera', frame)
    
    # Romper el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
